# Nescessary imports statements for the project lifecycle


In [2]:
!pip install contractions
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn
import torch.nn.functional as F
from nltk.corpus import stopwords 
from collections import Counter
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from wordcloud import WordCloud,STOPWORDS
import re,string,unicodedata
from nltk.corpus import stopwords
import plotly.express as px
from bs4 import BeautifulSoup
import contractions
from sklearn.feature_extraction.text import CountVectorizer

In [6]:
#Using torch to select device based on availability
is_cuda = torch.cuda.is_available()
if is_cuda:#If a GPU is available
    device = torch.device("cuda")
    print("GPU is available")
else:      #CPU
    device = torch.device("cpu")
    print("GPU not available, CPU used")

In [8]:
env = int(input('''
                    Enter 0 for kaggle
                    Enter 1 for google colab
                    Enter 2 for local machine'''))
base_path = '../input/nlpimdb/IMDB Dataset.csv'

if env == 0:   #Kaggle dev environment
    base_path = '../input/nlpimdb/IMDB Dataset.csv'
elif env == 1: #Google Colab
    from google.colab import drive
    drive.mount('/content/drive')
#   base_path = "/content/drive/MyDrive/imbd.csv"
    base_path = input('Enter your copied file path') 
elif env == 2: #Local Machine
#   base_path = 'IMDB Dataset.csv'
    base_path = input('Enter your copied file path') 

#Read data as pandas dataframe from csv
df = pd.read_csv(base_path)
df.head(5)

## Data Preprocessing, Feature Engineering and Exploratory Data Analysis

In [9]:
#Check for duplicates and drop if any
df = df.drop_duplicates(subset = ['review'], keep = 'first')
df.shape

In [10]:
#Check for missing value in the dataset. False meaing no missing value
df.isna().any().any()


In [11]:
eda_data = df.copy()

### Splitting to train and test data

 To avoid data lekage, We split our data train and test initiall. We split 80% for training and 20% as testing 

In [12]:
X,y = df['review'].values,df['sentiment'].values
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,stratify=y)
print(f'shape of train data is {x_train.shape}')
print(f'shape of test data is {x_test.shape}')

### Analysing the linguistic feature of the training set

As you can see from the graph that the given data is not unbalanced

### Tockenizing

In [13]:
def tockenizing(x_train,y_train,x_val,y_val):
    wordList = []
    stopWords = set(stopwords.words('english')) 
    for sent in x_train:
        for word in sent.lower().split():
            word = StringPreprocessing(word)
            if word not in stopWords and word != '':
                wordList.append(word)
  
    corpus = Counter(wordList)
    # sorting on the basis of most common words
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:1000]
    # creating a dict
    onehotDict = {w:i+1 for i,w in enumerate(corpus_)}
    
    # tockenizing
    fTrain,fTest = [],[]
    for sent in x_train:
            fTrain.append([onehotDict[StringPreprocessing(word)] for word in sent.lower().split() 
                                     if StringPreprocessing(word) in onehotDict.keys()])
    for sent in x_val:
            fTest.append([onehotDict[StringPreprocessing(word)] for word in sent.lower().split() 
                                    if StringPreprocessing(word) in onehotDict.keys()])
            
    encodedTrain = [1 if label =='positive' else 0 for label in y_train]  
    encodedTest = [1 if label =='positive' else 0 for label in y_val] 
    return np.array(fTrain), np.array(encodedTrain),np.array(fTest), np.array(encodedTest),onehotDict


def StringPreprocessing(s):
    # Remove all non-word characters (everything except numbers and letters)
    s = re.sub(r"[^\w\s]", '', s)
    # Replace all runs of whitespaces with no space
    s = re.sub(r"\s+", '', s)
    # replace digits with no space
    s = re.sub(r"\d", '', s)

    return s



In [14]:

x_train,y_train,x_test,y_test,vocab = tockenizing(x_train,y_train,x_test,y_test)

In [15]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text    


def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i.strip().lower().isalpha():
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def preprocess(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text = expand_contractions(text)
    return text


eda_data['review']=eda_data['review'].apply(preprocess)


In [16]:
# eda = df['review'].copy()
# data_cleaned = []
# seqlen = eda['review'].apply(lambda x: len(x.split()))
l = int(len(eda_data) * 0.8)
# train_fea = df[:l]
test_fea = eda_data[l:] #spliting to use


In [17]:
eda_data.sentiment.value_counts().sort_index(ascending=True).plot(kind='barh')

In [18]:
#Creating word cloud, Individualy for both positive and negative reviews using wordcloud module
def cloudOfWords(text):
    wordcloud = WordCloud(width=1600, height=800).generate(text)
    # Open a plot of the generated image.
    plt.figure( figsize=(20,10), facecolor='k')
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    plt.show()

print("WordCloud for postive reviews".center(120, " "))   
cloudOfWords(" ".join(eda_data[eda_data.sentiment == 'positive'].review)  )   
print("WordCloud for negative reviews".center(120, " "))   
cloudOfWords(" ".join(eda_data[eda_data.sentiment == 'negative'].review)  )   
    
    

In [19]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(12,8))
text_len=eda_data[eda_data['sentiment']=='positive']['review'].str.len()
ax1.hist(text_len,color='yellow')
ax1.set_title('Positive Reviews text')
text_len=eda_data[eda_data['sentiment']=='negative']['review'].str.len()
ax2.hist(text_len,color='red')
ax2.set_title('Negative Reviews text')
fig.suptitle('Characters in the text')
plt.show()

In [20]:
fig,(ax1,ax2)=plt.subplots(1,2,figsize=(20,10))
word=eda_data[eda_data['sentiment']=='positive']['review'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax1,color='yellow')
ax1.set_title('Positive reviews text')
word=eda_data[eda_data['sentiment']=='negative']['review'].str.split().apply(lambda x : [len(i) for i in x])
sns.distplot(word.map(lambda x: np.mean(x)),ax=ax2,color='red')
ax2.set_title('Negative reviews text')
fig.suptitle('Average word length in each review')

In [21]:
def get_corpus(text):
    words = []
    for i in text:
        for j in i.split():
            words.append(j.strip())
    return words
corpus = get_corpus(eda_data.review)
corpus[:5]

In [22]:
from collections import Counter
counter = Counter(corpus)
most_common = counter.most_common(10)
most_common = dict(most_common)
most_common

In [23]:
def get_top_text_ngrams(corpus, n, g):
    vec = CountVectorizer(ngram_range=(g, g)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [24]:
def ngrams(most_common):
    most_common_uni = dict(most_common)
    temp = pd.DataFrame(columns = ["Common_words" , 'Count'])
    temp["Common_words"] = list(most_common_uni.keys())
    temp["Count"] = list(most_common_uni.values())
    fig = px.bar(temp, x="Count", y="Common_words", title='Commmon Words in Text', orientation='h', 
             width=700, height=700,color='Common_words')
    fig.show()
ngrams(get_top_text_ngrams(eda_data.review,20,1))  
ngrams(get_top_text_ngrams(eda_data.review,20,2))    
ngrams(get_top_text_ngrams(eda_data.review,20,3))    

In [25]:
print(f'Length of vocabulary is {len(vocab)}')

### Analysing review length

In [26]:
rev_len = [len(i) for i in x_train]
pd.Series(rev_len).hist()
plt.show()
pd.Series(rev_len).describe()

### Padding

To maxLen each of the sequence will be padded

In [27]:
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

In [28]:
#we have fewer  number of reviews with length > 500.
#So we will consideronly those below it.
x_train_pad = padding_(x_train,500)
x_test_pad = padding_(x_test,500)


### Doing Batching and loading as tensor

In [29]:
# Creating Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
valid_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))
# DATALOADERS
batch_size = 50
# Shuffling data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=batch_size)

In [30]:
# A batch of training data
dataiter = iter(train_loader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print('Sample input: \n', sample_y)

### Model

In [31]:
class SentimentRNN(nn.Module):
    def __init__(self,no_layers,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5):
        super(SentimentRNN,self).__init__()
 
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
 
        self.no_layers = no_layers
        self.vocab_size = vocab_size
    
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm
        self.lstm = nn.LSTM(input_size=embedding_dim,hidden_size=self.hidden_dim,
                           num_layers=no_layers, batch_first=True)
        
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
    
        # linear and sigmoid layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size = x.size(0)
        # embeddings and lstm_out
        embeds = self.embedding(x)  # shape: B x S x Feature   since batch = True
        #print(embeds.shape)  #[50, 500, 1000]
        lstm_out, hidden = self.lstm(embeds, hidden)
        
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim) 
        
        # dropout and fully connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)

        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
        
        
        
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        h0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        c0 = torch.zeros((self.no_layers,batch_size,self.hidden_dim)).to(device)
        hidden = (h0,c0)
        return hidden

              

In [32]:
vocab_size = len(vocab) + 1 #extra 1 for padding
embedding_dim = 64
output_dim = 1
hidden_dim = 256
model = SentimentRNN(2,vocab_size,hidden_dim,embedding_dim,drop_prob=0.5)
model.to(device)#moving to gpu
print(model)#Model Summary

### Training process

In [33]:
# loss and optimization functions
lr=0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# function to predict accuracy
def acc(pred,label):
    pred = torch.round(pred.squeeze())
    return torch.sum(pred == label.squeeze()).item()

In [34]:
epochs = 5
clip = 5
valid_loss_min = np.Inf
# train for some number of epochs
epoch_tr_loss,epoch_vl_loss = [],[]
epoch_tr_acc,epoch_vl_acc = [],[]

for epoch in range(epochs):
    train_losses = []
    train_acc = 0.0
    model.train()
    # initialize hidden state 
    h = model.init_hidden(batch_size)
    for inputs, labels in train_loader:
        
        inputs, labels = inputs.to(device), labels.to(device)   
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])
        if( (inputs.shape[0],inputs.shape[1]) != (batch_size,500)):
            print("Validation - Input Shape Issue:",inputs.shape)
            continue
        model.zero_grad()
        output,h = model(inputs,h)
        
        # calculate the loss and perform backprop
        loss = criterion(output.squeeze(), labels.float())
        loss.backward()
        train_losses.append(loss.item())
        # calculating accuracy
        accuracy = acc(output,labels)
        train_acc += accuracy
        #`clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
 
    
        
    val_h = model.init_hidden(batch_size)
    val_losses = []
    val_acc = 0.0
    model.eval()
    for inputs, labels in valid_loader:
            val_h = tuple([each.data for each in val_h])

            inputs, labels = inputs.to(device), labels.to(device)
            if( (inputs.shape[0],inputs.shape[1]) != (batch_size,500)):
                print("Validation - Input Shape Issue:",inputs.shape)
                continue
            output, val_h = model(inputs, val_h)
            val_loss = criterion(output.squeeze(), labels.float())

            val_losses.append(val_loss.item())
            
            accuracy = acc(output,labels)
            val_acc += accuracy
            
    epoch_train_loss = np.mean(train_losses)
    epoch_val_loss = np.mean(val_losses)
    epoch_train_acc = train_acc/len(train_loader.dataset)
    epoch_val_acc = val_acc/len(valid_loader.dataset)
    epoch_tr_loss.append(epoch_train_loss)
    epoch_vl_loss.append(epoch_val_loss)
    epoch_tr_acc.append(epoch_train_acc)
    epoch_vl_acc.append(epoch_val_acc)
    print(f'Epoch {epoch+1}') 
    print(f'train_loss : {epoch_train_loss} val_loss : {epoch_val_loss}')
    print(f'train_accuracy : {epoch_train_acc*100} val_accuracy : {epoch_val_acc*100}')
    if epoch_val_loss <= valid_loss_min:
        torch.save(model.state_dict(), '../working/state_dict.pt')
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,epoch_val_loss))
        valid_loss_min = epoch_val_loss
    print(25*'==')
    

## Model Summary

In [35]:
fig = plt.figure(figsize = (20, 6))
plt.subplot(1, 2, 1)
plt.plot(epoch_tr_acc, label='Train Acc')
plt.plot(epoch_vl_acc, label='Validation Acc')
plt.title("Accuracy")
plt.legend()
plt.grid()
    
plt.subplot(1, 2, 2)
plt.plot(epoch_tr_loss, label='Train loss')
plt.plot(epoch_vl_loss, label='Validation loss')
plt.title("Loss")
plt.legend()
plt.grid()

plt.show()

### Inferance

In [36]:
def predict_text(text):
        word_seq = np.array([vocab[StringPreprocessing(word)] for word in text.split() 
                         if StringPreprocessing(word) in vocab.keys()])
        word_seq = np.expand_dims(word_seq,axis=0)
        pad =  torch.from_numpy(padding_(word_seq,500))
        inputs = pad.to(device)
        batch_size = 1
        h = model.init_hidden(batch_size)
        h = tuple([each.data for each in h])
        output, h = model(inputs, h)
        return(output.item())

In [37]:

index = 30
print(df['review'][index])
print('='*70)
print(f'Actual sentiment is  : {df["sentiment"][index]}')
print('='*70)
pro = predict_text("It's a sad movie but i liked it")
status = "positive" if pro > 0.51 else "negative"
pro = (1 - pro) if status == "negative" else pro
print(f'Predicted sentiment is {status} with a probability of {pro}')

In [38]:

index = 32
print(df['review'][index])
print('='*70)
print(f'Actual sentiment is  : {df["sentiment"][index]}')
print('='*70)
pro = predict_text(df['review'][index])
status = "positive" if pro > 0.5 else "negative"
pro = (1 - pro) if status == "negative" else pro
print(f'predicted sentiment is {status} with a probability of {pro}')

In [39]:
#Running model on Test review data
y_pred = []
for i in test_fea['review']:
    pro = predict_text(i)
    status = "positive" if pro > 0.5 else "negative"
    pro = (1 - pro) if status == "negative" else pro
    y_pred.append(status)


In [40]:
print(len(test_fea['sentiment']), len(y_pred))

In [41]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [42]:
# calculating confusion_matrix
y_test = test_fea['sentiment']
confusion_matrix(y_test, y_pred)

In [43]:
print(classification_report(y_test, y_pred)) # calculating precision, recall, f1 score

In [44]:
print('Accuracy score : ', accuracy_score(y_test, y_pred)) # Calculating model accuracy